In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import streamlit as st
from sentence_transformers import SentenceTransformer
import json

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Knowledge base - This would be replaced with actual CLAT data
clat_knowledge_base = [
    {
        "question": "What is the syllabus for CLAT 2025?",
        "answer": "The CLAT 2025 syllabus consists of five sections: English Language, Current Affairs including General Knowledge, Legal Reasoning, Logical Reasoning, and Quantitative Techniques. The English section tests reading comprehension, grammar, and vocabulary. Current Affairs includes general knowledge and current events. Legal Reasoning focuses on legal principles and their applications. Logical Reasoning tests analytical and critical thinking. Quantitative Techniques covers basic mathematics and data interpretation."
    },
    {
        "question": "How many questions are there in the English section?",
        "answer": "The English Language section in CLAT typically contains 28-32 questions. These questions are primarily based on reading comprehension passages, with some questions on grammar, vocabulary, and sentence completion."
    },
    {
        "question": "Give me last year's cut-off for NLSIU Bangalore.",
        "answer": "For the 2024 admission year, the cut-off for NLSIU Bangalore was approximately 116-118 marks out of 150 for general category students. For reserved categories, the cut-offs were lower: approximately 95-100 for OBC, 85-90 for SC, and 80-85 for ST. Please note that exact cut-offs may vary and you should verify with official sources."
    },
    {
        "question": "What is the CLAT exam pattern?",
        "answer": "The CLAT exam is a 2-hour test with 150 multiple-choice questions. Each question carries 1 mark, and there is a negative marking of 0.25 marks for each wrong answer. The exam is conducted online in English. The sections include English Language, Current Affairs & GK, Legal Reasoning, Logical Reasoning, and Quantitative Techniques."
    },
    {
        "question": "How should I prepare for the Legal Reasoning section?",
        "answer": "To prepare for the Legal Reasoning section, focus on understanding legal principles and their applications, rather than memorizing laws. Practice reading and analyzing legal passages, identifying principles, and applying them to scenarios. Study basic legal concepts, read legal news regularly, and solve previous years' papers. Books like 'Legal Aptitude for CLAT and other Law Entrance Examinations' by A.P. Bhardwaj can be helpful."
    },
    {
        "question": "What is the CLAT eligibility criteria?",
        "answer": "To be eligible for CLAT, candidates must have completed or be appearing for the 10+2 examination with a minimum of 45% marks (40% for SC/ST candidates). There is no upper age limit for the exam. Candidates in their final year of the qualifying exam can also apply, provided they produce proof of passing with the minimum required marks at the time of admission."
    },
    {
        "question": "Which are the top NLUs in India?",
        "answer": "The top National Law Universities (NLUs) in India, based on reputation and placement records, include NLSIU Bangalore, NALSAR Hyderabad, NUJS Kolkata, NLU Delhi, NLU Jodhpur, GNLU Gandhinagar, HNLU Raipur, RGNUL Patiala, and NUALS Kochi. Rankings may vary based on different parameters and sources."
    },
    {
        "question": "How many seats are available at NLSIU Bangalore?",
        "answer": "NLSIU Bangalore typically offers around 120 seats for its 5-year integrated B.A. LL.B. (Hons.) program. These seats are distributed across various categories including General, OBC, SC, ST, and others as per reservation policies. The exact number might vary slightly from year to year."
    },
    {
        "question": "What is the application fee for CLAT?",
        "answer": "The application fee for CLAT is ₹4,000 for General/OBC/PWD/NRI/PIO/OCI candidates and ₹3,500 for SC/ST/BPL candidates. The fee is payable online through various methods like credit card, debit card, or net banking."
    },
    {
        "question": "When is the CLAT 2025 exam date?",
        "answer": "The CLAT 2025 exam is tentatively scheduled for December 2024. However, the exact date will be announced by the Consortium of NLUs. It's advisable to regularly check the official CLAT website for the most updated information."
    },
    {
        "question": "How can I register for CLAT?",
        "answer": "To register for CLAT, visit the official website (consortiumofnlus.ac.in), create an account, fill in the application form with personal and academic details, upload required documents (photograph, signature, etc.), pay the application fee online, and submit the form. After submission, download and keep a copy of your application for future reference."
    }
]

# Preprocess the knowledge base
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and digits
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [w for w in tokens if w not in stop_words]
    # Lemmatize
    lemmatized = [lemmatizer.lemmatize(w) for w in filtered_tokens]
    return ' '.join(lemmatized)

# Prepare the knowledge base for TF-IDF
questions = [item["question"] for item in clat_knowledge_base]
answers = [item["answer"] for item in clat_knowledge_base]
preprocessed_questions = [preprocess_text(q) for q in questions]
combined_texts = [q + " " + a for q, a in zip(preprocessed_questions, answers)]

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(combined_texts)

# Load a pre-trained sentence transformer model for semantic search
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode questions for semantic search
question_embeddings = model.encode(questions)

# Function to get similar documents using TF-IDF
def get_tfidf_similar_documents(query, top_k=3):
    preprocessed_query = preprocess_text(query)
    query_vector = tfidf_vectorizer.transform([preprocessed_query])
    similarity_scores = cosine_similarity(query_vector, tfidf_matrix).flatten()
    top_indices = similarity_scores.argsort()[-top_k:][::-1]
    return [(answers[i], similarity_scores[i]) for i in top_indices]

# Function to get similar documents using semantic search
def get_semantic_similar_documents(query, top_k=3):
    query_embedding = model.encode([query])[0]
    similarities = cosine_similarity([query_embedding], question_embeddings)[0]
    top_indices = similarities.argsort()[-top_k:][::-1]
    return [(answers[i], similarities[i]) for i in top_indices]

# Function to generate a response based on the query
def get_response(query):
    # Get top results from both methods
    tfidf_results = get_tfidf_similar_documents(query)
    semantic_results = get_semantic_similar_documents(query)

    # Combine results (giving preference to semantic search)
    combined_results = semantic_results + [item for item in tfidf_results if item[0] not in [r[0] for r in semantic_results]]

    # Filter results with low similarity scores
    filtered_results = [item for item in combined_results if item[1] > 0.3]

    if filtered_results:
        # Return the answer with the highest score
        return filtered_results[0][0]
    else:
        # Direct string matching as a fallback
        query_lower = query.lower()
        keywords = query_lower.split()

        for item in clat_knowledge_base:
            # Check if any key terms match in the knowledge base questions
            if any(keyword in item["question"].lower() for keyword in keywords):
                return item["answer"]

        # If no match found, try to provide a generic response based on detected intent
        if "syllabus" in query_lower:
            return clat_knowledge_base[0]["answer"]
        elif "cut-off" in query_lower or "cutoff" in query_lower:
            return "Cut-off marks vary for each NLU and category. For specific information, please ask about a particular NLU."
        elif "exam pattern" in query_lower or "format" in query_lower:
            return clat_knowledge_base[3]["answer"]
        elif "prepare" in query_lower or "preparation" in query_lower:
            return "For CLAT preparation, focus on all five sections: English, Current Affairs & GK, Legal Reasoning, Logical Reasoning, and Quantitative Techniques. Regular practice with mock tests and previous years' papers is essential."

        return "I don't have specific information about that query. Please try rephrasing your question or ask about the CLAT syllabus, exam pattern, eligibility, or specific NLUs."

# Streamlit app
def main():
    st.title("CLAT Query Chatbot")
    st.write("Ask me anything about CLAT (Common Law Admission Test)")

    # Create a sidebar with info about the chatbot
    st.sidebar.title("About")
    st.sidebar.info("This chatbot provides information about the Common Law Admission Test (CLAT) including syllabus, exam pattern, eligibility, and more.")

    # Sample questions for users to try
    st.sidebar.title("Sample Questions")
    sample_questions = [
        "What is the syllabus for CLAT 2025?",
        "How many questions are there in the English section?",
        "What is the CLAT exam pattern?",
        "Give me last year's cut-off for NLSIU Bangalore",
        "How should I prepare for the Legal Reasoning section?",
        "What is the CLAT eligibility criteria?"
    ]

    for q in sample_questions:
        if st.sidebar.button(q):
            st.session_state.user_query = q

    # Initialize session state for chat history
    if 'chat_history' not in st.session_state:
        st.session_state.chat_history = []

    if 'user_query' not in st.session_state:
        st.session_state.user_query = ""

    # Display chat history
    for message in st.session_state.chat_history:
        if message['is_user']:
            st.write(f"You: {message['text']}")
        else:
            st.write(f"Bot: {message['text']}")

    # Input for the user query
    user_query = st.text_input("Your question:", value=st.session_state.user_query)

    if st.button("Ask") or st.session_state.user_query:
        if user_query:
            # Add user query to chat history
            st.session_state.chat_history.append({
                'text': user_query,
                'is_user': True
            })

            # Get response
            response = get_response(user_query)

            # Add bot response to chat history
            st.session_state.chat_history.append({
                'text': response,
                'is_user': False
            })

            # Clear the input field
            st.session_state.user_query = ""

            # Rerun to update the display
            st.experimental_rerun()

# Function to save the knowledge base to a file
def save_knowledge_base(knowledge_base, filename='clat_knowledge_base.json'):
    with open(filename, 'w') as f:
        json.dump(knowledge_base, f, indent=4)

# Function to load the knowledge base from a file
def load_knowledge_base(filename='clat_knowledge_base.json'):
    try:
        with open(filename, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        return []

# Option to save the initial knowledge base
save_knowledge_base(clat_knowledge_base)

if __name__ == "__main__":
    main()